In [1]:
import random
from typing import List, Dict
import os

In [2]:
class Student:
    def __init__(self, name: str):
        self.name = name

In [3]:
class Question:
    def __init__(self, text: str):
        self.text = text

    def __str__(self):
        return self.text

In [ ]:
class Topic:
    def __init__(self, name: str):
        self.name = name
        self.questions = []

    def add_question(self, question: Question):
        self.questions.append(question)

    def get_random_question(self):
        if not self.questions:
            raise ValueError("В теме нет доступных вопросов")
        question = random.choice(self.questions)
        self.questions.remove(question)
        return question

    def refresh_questions(self, questions: List[Question]):
        self.questions.extend(questions)

In [5]:
class Ticket:
    def __init__(self, student: Student, topics: Dict[str, Topic], question_counts: Dict[str, int]):
        self.student = student
        self.topics = topics
        self.question_counts = question_counts
        self.questions = self.generate_ticket()

    def generate_ticket(self):
        ticket_questions = {}
        for topic_name, count in self.question_counts.items():
            topic = self.topics.get(topic_name)
            if not topic:
                raise ValueError(f"Topic {topic_name} not found")
            ticket_questions[topic_name] = []
            for _ in range(count):
                if not topic.questions:
                    raise ValueError(f"Not enough questions in topic {topic_name}")
                question = topic.get_random_question()
                ticket_questions[topic_name].append(question)
            topic.refresh_questions(ticket_questions[topic_name])
        return ticket_questions

    def __str__(self):
        result = f"{self.student.name}\n"
        for topic, questions in self.questions.items():
            result += f"  {topic}:\n"
            for i, question in enumerate(questions, start=1):
                result += f"    {i}. {question}\n"
        return result

In [6]:
def read_students(file_path: str) -> List[Student]:
    with open(file_path, 'r', encoding='utf-8') as file:
        return [Student(line.strip()) for line in file]

def read_topics_and_questions(file_path: str) -> Dict[str, Topic]:
    topics = {}
    current_topic = None
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith("Тема:"):
                current_topic = Topic(line.split(":")[1].strip())
                topics[current_topic.name] = current_topic
            elif current_topic and line:
                current_topic.add_question(Question(line))
    return topics

In [ ]:
def main():
    students_file = input("Введите путь к файлу со студентами: ")
    topics_file = input("Введите путь к файлу с темами и вопросами: ")
    question_counts = {}

    while True:
        topic_name = input("Введите название темы (или 'стоп' для завершения): ")
        if topic_name.lower() == 'стоп':
            break
        count = int(input(f"Введите количество вопросов для темы '{topic_name}': "))
        question_counts[topic_name] = count

    students = read_students(students_file)
    topics = read_topics_and_questions(topics_file)

    with open("tickets.txt", 'w', encoding='utf-8') as file:
        for student in students:
            ticket = Ticket(student, topics, question_counts)
            file.write(str(ticket) + "\n")
            file.write("-" * 40 + "\n")

if __name__ == "__main__":
    main()